In [80]:
import numba
import numpy as np
import math
from numba import jit
from javelin.io import read_stru
from javelin.grid import Grid
from javelin.fourier_cython import calculate_cython, approx_calculate_cython

In [147]:
grid=Grid()


pzn = read_stru('pzn.stru')

qx1, qy1, qz1 = grid.get_squashed_q_meshgrid()
qx1 *= (2*np.pi)
qy1 *= (2*np.pi)
qz1 *= (2*np.pi)
qx1_2 = qx1*1j
qy1_2 = qy1*1j
qz1_2 = qz1*1j
qx2, qy2, qz2 = grid.get_q_meshgrid()
qx2 *= (2*np.pi)
qy2 *= (2*np.pi)
qz2 *= (2*np.pi)

qx3 = qx2*1j
qy3 = qy2*1j
qz3 = qz2*1j

positions = pzn.get_scaled_positions()[:1000]


Found a = 4.06, b = 4.06, c = 4.06, alpha = 90.0, beta = 90.0, gamma = 90.0
Read in these atoms:
O     375000
Pb    125000
Nb     83679
Zn     41321
Name: symbol, dtype: int64


In [97]:
print(qx1)

[[[ 0.        ]]

 [[ 0.06283185]]

 [[ 0.12566371]]

 [[ 0.18849556]]

 [[ 0.25132741]]

 [[ 0.31415927]]

 [[ 0.37699112]]

 [[ 0.43982297]]

 [[ 0.50265482]]

 [[ 0.56548668]]

 [[ 0.62831853]]

 [[ 0.69115038]]

 [[ 0.75398224]]

 [[ 0.81681409]]

 [[ 0.87964594]]

 [[ 0.9424778 ]]

 [[ 1.00530965]]

 [[ 1.0681415 ]]

 [[ 1.13097336]]

 [[ 1.19380521]]

 [[ 1.25663706]]

 [[ 1.31946891]]

 [[ 1.38230077]]

 [[ 1.44513262]]

 [[ 1.50796447]]

 [[ 1.57079633]]

 [[ 1.63362818]]

 [[ 1.69646003]]

 [[ 1.75929189]]

 [[ 1.82212374]]

 [[ 1.88495559]]

 [[ 1.94778745]]

 [[ 2.0106193 ]]

 [[ 2.07345115]]

 [[ 2.136283  ]]

 [[ 2.19911486]]

 [[ 2.26194671]]

 [[ 2.32477856]]

 [[ 2.38761042]]

 [[ 2.45044227]]

 [[ 2.51327412]]

 [[ 2.57610598]]

 [[ 2.63893783]]

 [[ 2.70176968]]

 [[ 2.76460154]]

 [[ 2.82743339]]

 [[ 2.89026524]]

 [[ 2.95309709]]

 [[ 3.01592895]]

 [[ 3.0787608 ]]

 [[ 3.14159265]]

 [[ 3.20442451]]

 [[ 3.26725636]]

 [[ 3.33008821]]

 [[ 3.39292007]]

 [[ 3.4557

In [98]:
print(qx2)

[[[ 0.        ]
  [ 0.        ]
  [ 0.        ]
  ..., 
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.06283185]
  [ 0.06283185]
  [ 0.06283185]
  ..., 
  [ 0.06283185]
  [ 0.06283185]
  [ 0.06283185]]

 [[ 0.12566371]
  [ 0.12566371]
  [ 0.12566371]
  ..., 
  [ 0.12566371]
  [ 0.12566371]
  [ 0.12566371]]

 ..., 
 [[ 6.1575216 ]
  [ 6.1575216 ]
  [ 6.1575216 ]
  ..., 
  [ 6.1575216 ]
  [ 6.1575216 ]
  [ 6.1575216 ]]

 [[ 6.22035345]
  [ 6.22035345]
  [ 6.22035345]
  ..., 
  [ 6.22035345]
  [ 6.22035345]
  [ 6.22035345]]

 [[ 6.28318531]
  [ 6.28318531]
  [ 6.28318531]
  ..., 
  [ 6.28318531]
  [ 6.28318531]
  [ 6.28318531]]]


In [99]:
print(qx3)

[[[ 0.+0.j        ]
  [ 0.+0.j        ]
  [ 0.+0.j        ]
  ..., 
  [ 0.+0.j        ]
  [ 0.+0.j        ]
  [ 0.+0.j        ]]

 [[ 0.+0.06283185j]
  [ 0.+0.06283185j]
  [ 0.+0.06283185j]
  ..., 
  [ 0.+0.06283185j]
  [ 0.+0.06283185j]
  [ 0.+0.06283185j]]

 [[ 0.+0.12566371j]
  [ 0.+0.12566371j]
  [ 0.+0.12566371j]
  ..., 
  [ 0.+0.12566371j]
  [ 0.+0.12566371j]
  [ 0.+0.12566371j]]

 ..., 
 [[ 0.+6.1575216j ]
  [ 0.+6.1575216j ]
  [ 0.+6.1575216j ]
  ..., 
  [ 0.+6.1575216j ]
  [ 0.+6.1575216j ]
  [ 0.+6.1575216j ]]

 [[ 0.+6.22035345j]
  [ 0.+6.22035345j]
  [ 0.+6.22035345j]
  ..., 
  [ 0.+6.22035345j]
  [ 0.+6.22035345j]
  [ 0.+6.22035345j]]

 [[ 0.+6.28318531j]
  [ 0.+6.28318531j]
  [ 0.+6.28318531j]
  ..., 
  [ 0.+6.28318531j]
  [ 0.+6.28318531j]
  [ 0.+6.28318531j]]]


In [148]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
calculate_cython(qx2, qy2, qz2, positions, temp_array.real, temp_array.imag)
print(temp_array.sum())

(-127267.422558+922234.458939j)


In [149]:
cex = np.exp(np.linspace(0, 2j*np.pi*(1-2**-16), 2**16))
temp_array = np.zeros(grid.bins, dtype=np.complex)
approx_calculate_cython(grid.ll, grid.v1_delta, grid.v2_delta, grid.v3_delta, positions, temp_array.real, temp_array.imag, cex.real, cex.imag)
print(temp_array.sum())

(-127227.326558+922292.207797j)


In [150]:
def f1(positions, temp_array, qx, qy, qz):
    for atom in positions:
        temp_array += np.exp(qx*atom[0]*1j) * np.exp(qy*atom[1]*1j) * np.exp(qz*atom[2]*1j)

In [151]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f1(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

(-127267.422558+922234.458939j)


In [152]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [105]:
%lprun -f f1 f1(positions, temp_array, qx1, qy1, qz1)

In [153]:
def f1_2(positions, temp_array, qx, qy, qz):
    for atom in positions:
        temp_array += np.exp(qx*atom[0]) * np.exp(qy*atom[1]) * np.exp(qz*atom[2])

In [154]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f1_2(positions, temp_array, qx1_2, qy1_2, qz1_2)
print(temp_array.sum())

(-127267.422558+922234.458939j)


In [108]:
%lprun -f f1_2 f1_2(positions, temp_array, qx1_2, qy1_2, qz1_2)

In [155]:
def f2(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dot = qx*atom[0] + qy*atom[1] + qz*atom[2]
        temp_array += np.exp(dot*1j)

In [156]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f2(positions, temp_array, qx2, qy2, qz2)
print(temp_array.sum())

(-127267.422558+922234.458939j)


In [111]:
%lprun -f f2 f2(positions, temp_array, qx2, qy2, qz2)

In [112]:
def f3(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dot = qx*atom[0] + qy*atom[1] + qz*atom[2]
        temp_array += np.exp(dot)

In [157]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f3(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

(-127267.422558+922234.458939j)


In [114]:
%lprun -f f3 f3(positions, temp_array, qx3, qy3, qz3)

In [115]:
def f4(positions, temp_array, qx, qy, qz):
    N = positions.shape[0]
    A, B, C = temp_array.shape
    for n in range(N):
        for i in range(A):
            for j in range(B):
                for k in range(C):
                    temp_array[i,j,k] += np.exp(qx[i,j,k]*positions[n,0] + qy[i,j,k]*positions[n,1] + qz[i,j,k]*positions[n,2])

In [158]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f4(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

(-127267.422558+922234.458939j)


In [117]:
%lprun -f f4 f4(positions, temp_array, qx3, qy3, qz3)

In [170]:
f1_jit = jit()(f1)
f1_2_jit = jit()(f1_2)
f2_jit = jit(cache=True)(f2)
f3_jit = jit(cache=True)(f3)
f4_jit = jit()(f4)

In [159]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f1(positions, temp_array, qx1, qy1, qz1)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f1_jit(positions, temp_array, qx1, qy1, qz1)

183 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
175 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [160]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit calculate_cython(qx2, qy2, qz2, positions, temp_array.real, temp_array.imag)

56.4 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [161]:
cex = np.exp(np.linspace(0, 2j*np.pi*(1-2**-16), 2**16))
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit approx_calculate_cython(grid.ll, grid.v1_delta, grid.v2_delta, grid.v3_delta, positions, temp_array.real, temp_array.imag, cex.real, cex.imag)

8.5 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [169]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f1_2(positions, temp_array, qx1_2, qy1_2, qz1_2)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f1_2_jit(positions, temp_array, qx1_2, qy1_2, qz1_2)

161 ms ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
162 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [171]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f2(positions, temp_array, qx2, qy2, qz2)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f2_jit(positions, temp_array, qx2, qy2, qz2)

961 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
956 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [164]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f3(positions, temp_array, qx3, qy3, qz3)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f3_jit(positions, temp_array, qx3, qy3, qz3)

987 ms ± 866 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
985 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [165]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f4(positions, temp_array, qx2, qy2, qz2)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f4_jit(positions, temp_array, qx2, qy2, qz2)

1min 9s ± 3.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
338 ms ± 631 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [166]:
f4_jit_np = jit(nopython=True)(f4)

In [167]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f4_jit_np(positions, temp_array, qx2, qy2, qz2)

337 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [127]:
f1_jit.inspect_types()

f1 (array(float64, 2d, A), array(complex128, 3d, C), array(float64, 3d, C), array(float64, 3d, C), array(float64, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-102-d0c2edfd87c9>
# --- LINE 1 --- 
# label 0

def f1(positions, temp_array, qx, qy, qz):

    # --- LINE 2 --- 
    #   positions = arg(0, name=positions)  :: pyobject
    #   temp_array = arg(1, name=temp_array)  :: pyobject
    #   qx = arg(2, name=qx)  :: pyobject
    #   qy = arg(3, name=qy)  :: pyobject
    #   qz = arg(4, name=qz)  :: pyobject
    #   jump 2
    # label 2
    #   $52 = const(LiftedLoop, LiftedLoop(<function f1 at 0x7f54287738c8>))  :: XXX Lifted Loop XXX
    #   $53 = call $52(positions, qx, qy, qz, temp_array, func=$52, args=[Var(positions, <ipython-input-102-d0c2edfd87c9> (2)), Var(qx, <ipython-input-102-d0c2edfd87c9> (2)), Var(qy, <ipython-input-102-d0c2edfd87c9> (2)), Var(qz, <ipython-input-102-d0c2edfd87c9> (2)), Var(temp_array, <ipyth

In [128]:
f2_jit.inspect_types()

f2 (array(float64, 2d, A), array(complex128, 3d, C), array(float64, 3d, C), array(float64, 3d, C), array(float64, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-109-783fedab7279>
# --- LINE 1 --- 
# label 0

def f2(positions, temp_array, qx, qy, qz):

    # --- LINE 2 --- 
    #   positions = arg(0, name=positions)  :: pyobject
    #   temp_array = arg(1, name=temp_array)  :: pyobject
    #   qx = arg(2, name=qx)  :: pyobject
    #   qy = arg(3, name=qy)  :: pyobject
    #   qz = arg(4, name=qz)  :: pyobject
    #   jump 2
    # label 2
    #   $44 = const(LiftedLoop, LiftedLoop(<function f2 at 0x7f542838c840>))  :: XXX Lifted Loop XXX
    #   $45 = call $44(positions, qx, qy, qz, temp_array, func=$44, args=[Var(positions, <ipython-input-109-783fedab7279> (2)), Var(qx, <ipython-input-109-783fedab7279> (2)), Var(qy, <ipython-input-109-783fedab7279> (2)), Var(qz, <ipython-input-109-783fedab7279> (2)), Var(temp_array, <ipyth

In [129]:
f3_jit.inspect_types()

f3 (array(float64, 2d, A), array(complex128, 3d, C), array(complex128, 3d, C), array(complex128, 3d, C), array(complex128, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-112-ca0baffb8abb>
# --- LINE 1 --- 
# label 0

def f3(positions, temp_array, qx, qy, qz):

    # --- LINE 2 --- 
    #   positions = arg(0, name=positions)  :: pyobject
    #   temp_array = arg(1, name=temp_array)  :: pyobject
    #   qx = arg(2, name=qx)  :: pyobject
    #   qy = arg(3, name=qy)  :: pyobject
    #   qz = arg(4, name=qz)  :: pyobject
    #   jump 2
    # label 2
    #   $42 = const(LiftedLoop, LiftedLoop(<function f3 at 0x7f542838cd90>))  :: XXX Lifted Loop XXX
    #   $43 = call $42(positions, qx, qy, qz, temp_array, func=$42, args=[Var(positions, <ipython-input-112-ca0baffb8abb> (2)), Var(qx, <ipython-input-112-ca0baffb8abb> (2)), Var(qy, <ipython-input-112-ca0baffb8abb> (2)), Var(qz, <ipython-input-112-ca0baffb8abb> (2)), Var(temp_arra

In [130]:
f4_jit.inspect_types()

f4 (array(float64, 2d, A), array(complex128, 3d, C), array(float64, 3d, C), array(float64, 3d, C), array(float64, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-115-7ea15ab547fa>
# --- LINE 1 --- 
# label 0
#   del $const0.3
#   del $0.2
#   del $0.4
#   del $0.6
#   del $0.10
#   del $0.7
#   del $0.8
#   del $0.9

def f4(positions, temp_array, qx, qy, qz):

    # --- LINE 2 --- 
    #   positions = arg(0, name=positions)  :: array(float64, 2d, A)
    #   temp_array = arg(1, name=temp_array)  :: array(complex128, 3d, C)
    #   qx = arg(2, name=qx)  :: array(float64, 3d, C)
    #   qy = arg(3, name=qy)  :: array(float64, 3d, C)
    #   qz = arg(4, name=qz)  :: array(float64, 3d, C)
    #   $0.2 = getattr(value=positions, attr=shape)  :: (int64 x 2)
    #   $const0.3 = const(int, 0)  :: int64
    #   $0.4 = static_getitem(value=$0.2, index=0, index_var=$const0.3)  :: int64
    #   N = $0.4  :: int64

    N = positions.sha

In [131]:
f1_jit_np = jit(nopython=True,parallel=True)(f1)

In [132]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f1_jit_np(positions, temp_array, qx1, qy1, qz1)

LoweringError: Failed at nopython (nopython mode backend)
iterating over 2D array
File "<ipython-input-102-d0c2edfd87c9>", line 2
[1] During: lowering "$6.2 = iternext(value=$2.2)" at <ipython-input-102-d0c2edfd87c9> (2)

In [133]:
def f5(positions, temp_array, qx, qy, qz):
    N = positions.shape[0]
    for n in range(N):
        temp_array += np.exp(qx*positions[n,0] + qy*positions[n,1] + qz*positions[n,2])

In [134]:
f5_jit = jit()(f5)
f5_jit_np = jit(nopython=True)(f5)
f5_jit_np_p = jit(nopython=True,parallel=True)(f5)

In [135]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f5(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

(45992.5552559+289015.840671j)


In [136]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f5_jit(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

(45992.5552559+289015.840671j)


In [137]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f5_jit_np(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

(45992.5552559+289015.840671j)


In [138]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f5_jit_np_p(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

(nan+3.45368545185e+299j)


In [139]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f5(positions, temp_array, qx3, qy3, qz3)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f5_jit(positions, temp_array, qx3, qy3, qz3)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f5_jit_np(positions, temp_array, qx3, qy3, qz3)
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f5_jit_np_p(positions, temp_array, qx3, qy3, qz3)

92.7 ms ± 33 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
116 ms ± 70.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
114 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
35.3 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [140]:
f5_jit_np_p.inspect_types()

f5 (array(float64, 2d, A), array(complex128, 3d, C), array(complex128, 3d, C), array(complex128, 3d, C), array(complex128, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-133-e849fef06982>
# --- LINE 1 --- 
# label 0
#   del temp_array_shape.304
#   del qx_shape.308
#   del qy_shape.312
#   del qz_shape.316
#   del $const0.3
#   del $0.2
#   del $12.1
#   del $0.4
#   del $12.3

def f5(positions, temp_array, qx, qy, qz):

    # --- LINE 2 --- 
    #   positions = arg(0, name=positions)  :: array(float64, 2d, A)
    #   temp_array = arg(1, name=temp_array)  :: array(complex128, 3d, C)
    #   temp_array_shape.304 = getattr(value=temp_array, attr=shape)  :: (int64 x 3)
    #   temp_array_size0.305 = static_getitem(value=temp_array_shape.304, index=0, index_var=None)  :: int64
    #   temp_array_size1.306 = static_getitem(value=temp_array_shape.304, index=1, index_var=None)  :: int64
    #   temp_array_size2.307 = static_geti

In [141]:
@jit(nopython=True, parallel=True)
def f6_exp(position,temp_array,qx,qy,qz):
    temp_array += np.exp(qx*positions[0] + qy*positions[1] + qz*positions[2])

def f6(positions, temp_array, qx, qy, qz):
    N = positions.shape[0]
    for n in range(N):
        f6_exp(positions[n], temp_array, qx, qy, qz)

In [142]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f6(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

AssertionError: Sizes of qx, $0.7, qy, $0.12, qz, $0.18 do not match

In [ ]:
def f7(positions, temp_array, qx, qy, qz):
    N = positions.shape[0]
    for n in range(N):
        dotx = np.exp(qx*positions[n,0]*1j)
        doty = np.exp(qy*positions[n,1]*1j)
        dotz = np.exp(qz*positions[n,2]*1j)
        temp_array += dotx * doty * dotz

In [ ]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f7(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

In [ ]:
f7_jit=jit()(f7)
f7_jit_np=jit(nopython=True)(f7)
f7_jit_np_p=jit(nopython=True,parallel=True)(f7)

In [ ]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f7(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

In [ ]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f7_jit(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

In [ ]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f7_jit_np(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

In [ ]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
f7_jit_np_p(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())